In [1]:
import torch
import torch.nn.functional as F
import string
import gradio as gr

In [2]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

all_categories = ['Arabic','Chinese','Czech','Dutch','English','French','German','Greek',
                'Irish','Italian','Japanese','Korean','Polish','Portuguese','Russian','Scottish',
                'Spanish','Vietnamese']

In [3]:
# Find letter index from all_letters: Ex: "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

In [4]:
# Giving each charachter in name a one hot vector
def lineToTensor(line):
    tensor = torch.zeros(len(line),1,n_letters)
    for li,letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1

    return tensor 

In [5]:
# Loading in torchscript model
my_model = torch.jit.load('name_classifier_ts.ptl')

In [6]:
# Return output given a line_tensor
def evaluate(line_tensor):
    hidden = torch.zeros(1,128)

    for i in range(line_tensor.size()[0]):
        output, hidden = my_model(line_tensor[i], hidden)

    return output

In [7]:
# Feeding in a name and number of top predictions you want to output
def predict(last_name,n_predictions=3):
    
    last_name = last_name.title()
    with torch.no_grad():
        output = evaluate(lineToTensor(last_name))
        output = F.softmax(output,dim=1)

        topv,topi = output.topk(n_predictions,1,True)
        
        top_3_countries = ''
        for i in range(n_predictions):
            value = topv[0]
            category_index = topi[0][i].item()
            top_3_countries += f'{all_categories[category_index]}: {round(value[i].item()*100,2)}%'
            top_3_countries += '\n'
    return top_3_countries

In [8]:
out = predict('Flores')

In [9]:
out

'Spanish: 37.8%\nPortuguese: 37.2%\nDutch: 7.46%\n'

In [10]:
demo = gr.Interface(predict,
                        inputs = "text", 
                        outputs = "text", 
                        description="Classify name into language of origin. Returns top 3 languages of origin"
                    )

In [11]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://33368.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x1f2f05ed610>,
 'http://127.0.0.1:7860/',
 'https://33368.gradio.app')